# jobflow

[`jobflow`](https://materialsproject.github.io/jobflow/index.html) and [`atomate2`](https://materialsproject.github.io/atomate2/index.html) are key packages of the [Materials Project](https://materialsproject.org/) . `jobflow` was especially designed to simplify the execution of dynamic workflows -- when the actual number of jobs is dynamically determined upon runtime instead of being statically fixed before running the workflow(s). `jobflow`'s overall flexibility allows for building workflows that go beyond the usage in materials science. `jobflow` serves as the basis of `atomate2`, which implements data generation workflows in the context of materials science and will be used for data generation in the Materials Project in the future.

## Define workflow with jobflow

We start by importing the job decorator and Flow class from `jobflow` and the respective PWD tools.

In [ ]:
import numpy as np

In [ ]:
from jobflow import job, Flow

In [ ]:
from python_workflow_definition.jobflow import write_workflow_json

## Quantum Espresso Workflow
We will use the knowledge from the previous arithmetic workflow example to create the Quantum Espresso-related tasks for calculating an "Energy vs. Volume" curve. It’s important to note that this is only a basic implementation, and further extensions towards data validation or for a simplified user experience can be added. For example, one can typically configure run commands for quantum-chemical programs via configuration files in atomate2.

In [ ]:
from workflow import (
    calculate_qe as _calculate_qe, 
    generate_structures as _generate_structures, 
    get_bulk_structure as _get_bulk_structure, 
    plot_energy_volume_curve as _plot_energy_volume_curve,
)

In [ ]:
workflow_json_filename = "jobflow_qe.json"

In [ ]:
calculate_qe = job(_calculate_qe)
generate_structures = job(_generate_structures)
plot_energy_volume_curve = job(_plot_energy_volume_curve)
get_bulk_structure = job(_get_bulk_structure)

We need to specify the typical QE input like pseudopotential(s) and structure model.

In [ ]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [ ]:
structure = get_bulk_structure(
    element="Al",
    a=4.04,
    cubic=True,
)

In [ ]:
calc_mini = calculate_qe(
    working_directory="mini",
    input_dict={
        "structure": structure.output,
        "pseudopotentials": pseudopotentials,
        "kpts": (3, 3, 3),
        "calculation": "vc-relax",
        "smearing": 0.02,
    },
)

Next, for the "Energy vs. Volume" curve, we meed to specify the number of strained structures and save them into a list object. For each of the strained structures, we will carry out a QE calculation.

In [ ]:
number_of_strains = 5
structure_lst = generate_structures(
    structure=calc_mini.output.structure,
    strain_lst=np.linspace(0.9, 1.1, number_of_strains),
)

In [ ]:
job_strain_lst = []
for i in range(number_of_strains):
    calc_strain = calculate_qe(
        working_directory="strain_" + str(i),
        input_dict={
            "structure": getattr(structure_lst.output, f"s_{i}"),
            "pseudopotentials": pseudopotentials,
            "kpts": (3, 3, 3),
            "calculation": "scf",
            "smearing": 0.02,
        },
    )
    job_strain_lst.append(calc_strain)

Finally, we specify a plotter for the "Energy vs. Volume" curve and can export the workflow.

In [ ]:
plot = plot_energy_volume_curve(
    volume_lst=[job.output.volume for job in job_strain_lst],
    energy_lst=[job.output.energy for job in job_strain_lst],
)

In [ ]:
flow = Flow([structure, calc_mini, structure_lst] + job_strain_lst + [plot])

In [ ]:
write_workflow_json(flow=flow, file_name=workflow_json_filename)

In [ ]:
!cat {workflow_json_filename}

## Load Workflow with aiida

Now, we can import the workflow, run it with `aiida` and plot the "Energy vs. Volume" curve.

In [ ]:
from aiida import orm, load_profile

load_profile()

In [ ]:
from python_workflow_definition.aiida import load_workflow_json

In [ ]:
wg = load_workflow_json(workflow_json_filename)

In [ ]:
wg.nodes.get_bulk_structure1.inputs.a.value = orm.Float(4.05)

In [ ]:
wg

In [ ]:
wg.run()

## Load Workflow with pyiron_base

And we can repeat the same process using `pyiron`.

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
delayed_object_lst = load_workflow_json(file_name=workflow_json_filename)
delayed_object_lst[-1].draw()

In [ ]:
delayed_object_lst[0].input['a'] = 4.05

In [ ]:
delayed_object_lst[-1].pull()

## Load Workflow with pyiron_workflow

In [ ]:
from python_workflow_definition.pyiron_workflow import load_workflow_json

In [ ]:
wf = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
wf.get_bulk_structure.inputs.a.value = 4.05

In [ ]:
wf.draw(size=(10,10))

In [ ]:
wf.run()